## Requirements

In [ ]:
import requests
import pandas as pd
import datetime
import os
import numpy as np

In [ ]:
api_key = os.environ.get('EOD_Historical_Data_API_Key')

## ETL Functions

In [ ]:
def getDailyStockPrices(symbol: str, start_date: str, end_date: str = None):

    #Set last date of dataset to today unless assigned during function call
    if end_date is None:
        end_date = datetime.datetime.now().strftime("%Y-%m-%d")

    # Validate Start & End Date 
    # Convert strings to datetime objects
    start_date_dtObj = datetime.datetime.strptime(start_date, "%Y-%m-%d")
    end_date_dtObj = datetime.datetime.strptime(end_date, "%Y-%m-%d")
    # Validate
    end_date = max(end_date_dtObj, start_date_dtObj)
    start_date = min(end_date_dtObj, start_date_dtObj)

    # API Request to return Pandas DataFrame
    api_url = f'https://eodhistoricaldata.com/api/eod/{symbol}.US?from={start_date}&to={end_date}&api_token={api_key}&fmt=json'
    data = requests.get(api_url).json()
    df = pd.DataFrame(data)

    # Add column to DataFrame containing Symbol
    df.insert(loc = 0, column = 'symbol', value = symbol)

    # Format date column as datetime, and set as DataFrame index
    df['date'] = pd.to_datetime(df['date'])
    df = df.set_index("date", drop=False)

    return df

In [ ]:
def getIntradayStockPrices(symbol: str, interval: int = 2):

    # API Request to return Pandas DataFrame
    api_url = f'https://eodhistoricaldata.com/api/intraday/{symbol}.US?api_token={api_key}&interval=1m&fmt=json'
    data = requests.get(api_url).json()
    df = pd.DataFrame(data)

    # Convert UNIX datetime from UTC to Eastern
    unixUTC = pd.to_datetime(df['timestamp'], unit='s')
    df['datetime_est'] = unixUTC.dt.tz_localize('UTC').dt.tz_convert('US/Eastern').dt.tz_localize(None)
    
    # Add column to DataFrame, in 1st position, containing truncated Date (EST)
    df.insert(loc = 0, column = 'date_est', value = df['datetime_est'].dt.date)

    # Define 'Trading Session' function **LOOK INTO VECTORIZING**
    def tradingSession(time):
        # Define 'Trading Session' variables
        marketStart = datetime.time(9, 30, 0)
        marketEnd = datetime.time(16, 0, 0) 
        # Define 'Trading Session' logic       
        if time < marketStart:
            return "Pre-Market"
        elif time >= marketEnd:
            return "After Hours"
        else:
            return "Market"

    # Calculate 'Trading Session' for all data points
    df['trading_session'] = df['datetime_est'].dt.time.apply(tradingSession)
   
    # Filter for Market hours
    df = df[df['trading_session'] == 'Market']

    # Define Interval Groups
    position = df.columns.get_loc('datetime_est')
    elapsed = df.iloc[0:, position] - df.iat[0, position]
    minutes_elapsed = (elapsed.dt.seconds/60) + 1
    df['interval_group'] = -(-minutes_elapsed // (390/interval)) # Upside-down floor division to convert floor to ceiling with negation. 

    # Aggregate Data using Interval Groups
    agg_dict = {
        'open': 'first',
        'high': np.max,
        'low': np.min,
        'close': 'last',
        'volume': np.sum
    }
    df = df.groupby(['date_est', 'interval_group']).agg(agg_dict).reset_index()

    # Add column to DataFrame, in 1st position, containing Symbol
    df.insert(loc = 0, column = 'symbol', value = symbol)

    return df